# Segmentez des clients d'un site e-commerce - *Notebook simulation*

In [4]:
import numpy as np

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FuncFormatter
import scipy
from scipy import stats
import scipy.stats as st

import statsmodels
import statsmodels.api as sm
import missingno as msno

import sklearn

from sklearn.impute import KNNImputer
from category_encoders.ordinal import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer
from category_encoders import TargetEncoder

# pour le centrage et la réduction
from sklearn.preprocessing import StandardScaler
# pour l'ACP
from sklearn.decomposition import PCA

from sklearn import model_selection
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.datasets import make_blobs

from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier

from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

from sklearn.manifold import TSNE

from sklearn.utils import resample

from yellowbrick.cluster import KElbowVisualizer

from scipy.cluster.hierarchy import dendrogram, linkage

import timeit
import warnings

# pour cartographie
import folium
import cartopy.crs as ccrs
import geopandas as gpd
import contextily as ctx

print("numpy version", np.__version__)
print("pandas version", pd.__version__)
print("matplotlib version", matplotlib.__version__)
print("seaborn version", sns.__version__)
print("scipy version", scipy.__version__)
print("statsmodels version", statsmodels.__version__)
print("missingno version", msno.__version__)

print("sklearn version", sklearn.__version__)

print("folium version", folium.__version__)

pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

numpy version 1.26.4
pandas version 2.1.4
matplotlib version 3.8.0
seaborn version 0.13.2
scipy version 1.11.4
statsmodels version 0.14.0
missingno version 0.5.2
sklearn version 1.2.2
folium version 0.19.2


## 1 - Démarche

J'utiliserais les mêmes features que celles utilisées dans le 2ème essai; c'est à dire :
- récence
- fréquence
- montant
- score_moyen

Voici les étapes :
- **T0 :** entraînement initial du modèle M0 sur le dataset X_0, créé à partir de toutes les commandes passées jusqu’à T0.
    - L'entraînement de ce modème génére la liste des clusters C0=M0.fit_predict(X_0).
- **T1 :**  entrainement du nouveau modème M1 sur le dataset X_1, créé à partir de toutes les commandes passées jusqu’à T1.
    - L'entraînement de ce modème génére la liste des clusters C1_new=M1.fit_predict(X_1).
    - On fait ensuite une nouvelle prédiction avec M0 sur M1 ==> C1_init = M0.predict(X_1).
    - On calcule l'ARI entre C1 et C1_init avec adjusted_rand_score avec sklearn.
- **T2 :**  entrainement du nouveau modème M2 sur le dataset X_2, créé à partir de toutes les commandes passées jusqu’à T2.
    - L'entraînement de ce modème génére la liste des clusters C2_new=M2.fit_predict(X_2).
    - On fait ensuite une nouvelle prédiction avec M0 sur M2 ==> C2_init = M0.predict(X_2).
    - On calcule l'ARI entre C2_new et C2_init avec adjusted_rand_score avec sklearn.         

## 2 - Préparation du jeu de données

### 2.1 - Chargement du jeu de données

J'utilise un accès direct à la base de données olist.db.

La fonction créée pour extraire les données de la base prends le paramètre date_select (T0, T1, T2,...)

In [16]:
import sqlite3

# Connexion à une base SQLite (ou création si elle n'existe pas)
conn = sqlite3.connect('C:/Users/admin/Documents/Projets/Projet_5/olist.db')

# Créer un curseur
cursor = conn.cursor()

# Exécuter une requête
cursor.execute("WITH latest_order AS(\
                	SELECT max(order_purchase_timestamp) AS max_purchase_date\
                FROM orders\
                ),\
                orders_join_rfm AS(\
                	SELECT c.customer_unique_id,\
                	       CAST(julianday((SELECT max_purchase_date FROM latest_order)) - julianday(MAX(o.order_purchase_timestamp)) AS INTEGER) AS recence,\
                    	   COUNT(DISTINCT o.order_id) AS frequence,\
                    	   SUM(oi.price) AS montant\
                	    FROM customers c\
                	    LEFT JOIN orders o ON c.customer_id = o.customer_id\
                	    LEFT JOIN order_items oi ON o.order_id = oi.order_id\
                		GROUP BY c.customer_unique_id\
                ),\
                review_joins AS(\
                	SELECT c.customer_unique_id, AVG(or2.review_score) AS score_moyen\
                		FROM customers c\
                		INNER JOIN orders o ON o.customer_id = c.customer_id\
                		INNER JOIN order_reviews or2 ON or2.order_id = o.order_id\
                		GROUP BY c.customer_unique_id \
                ),\
                aggregation AS(\
                	SELECT orfm.customer_unique_id, recence, frequence, montant, score_moyen\
                		FROM orders_join_rfm orfm\
                		LEFT JOIN review_joins rj ON rj.customer_unique_id = orfm.customer_unique_id\
                )\
                SELECT * \
                	FROM aggregation")
rows = cursor.fetchall()

# Fermer la connexion
conn.close()

Chargeons les lignes dans un dataframe :

In [29]:
# Obtenir les noms de colonnes
columns = [description[0] for description in cursor.description]

# Convertir en DataFrame
df = pd.DataFrame(rows, columns=["customer_unique_id", "recence", "frequence", "montant", "score_moyen"])

df.head()

,customer_unique_id,recence,frequence,montant,score_moyen
0,0000366f3b9a7992bf8c76cfdf3221e2,160,1,129.90,5.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,163,1,18.90,4.0
2,0000f46a3911fa3c0805444483337064,585,1,69.00,3.0
3,0000f6ccb0745a6a4b88665a16c9f078,369,1,25.99,4.0
4,0004aac84e0df4da2b147fca70cf8255,336,1,180.00,5.0
